In [13]:
from ClosestNeighbor import ClosestNeighbor
from NeighborsResearch import NeighborsResearch
from VND import VND

from tqdm import tqdm
from datetime import datetime
import pandas as pd
import statistics

# Construção da Tabela

## Constantes

In [2]:
f = open("instancias_teste/otimos", "r")
otimos = {}
for x in f:
  line = x.split(" ")
  otimos[line[0]] = int(line[2])

execucoes = 10
vnd = VND()
tabela = pd.DataFrame(index = otimos.keys(), columns = ["otimo", "HC - Média solução", 
"HC - Melhor Solução", "HC - Média Tempo", "HC - gap", "(HC + VND) - Média solução", 
"(HC + VND) - Melhor Solução", "(HC + VND) - Média Tempo", "(HC + VND) - gap"])

## Execução do algoritmo e preenchimento da tabela

In [14]:
for situation in tqdm(otimos):
    tabela["otimo"][situation] = otimos[situation]
    solucoes_time_HC = []
    solucoes_time_HC_VND = []
    solucoes_HC = []
    solucoes_HC_VND = []
    for i in range(execucoes):
        HeuristicAlgorithm = ClosestNeighbor(situation)
        now_HC_VND = datetime.now()
        now_HC = datetime.now()
        HeuristicAlgorithm.closestNeighbor()
        later_HC = datetime.now()
        solucoes_time_HC.append((later_HC - now_HC).total_seconds())
        
        paths = HeuristicAlgorithm.get_truck_path()
        total_distances = HeuristicAlgorithm.get_truck_total_distance()
        matrix = HeuristicAlgorithm.get_distance_matrix()
        solucoes_HC.append(sum(total_distances.values()))
        
        new_paths, new_total_distances = vnd.run_vnd(paths, total_distances, matrix)
        later_HC_VND = datetime.now()
        solucoes_HC_VND.append(sum(new_total_distances.values()))
        solucoes_time_HC_VND.append((later_HC_VND - now_HC_VND).total_seconds())
        
    tabela["HC - Média Tempo"][situation] = statistics.mean(solucoes_time_HC)
    tabela["HC - Média solução"][situation] = statistics.mean(solucoes_HC)
    tabela["HC - Melhor Solução"][situation] = sorted(solucoes_HC)[0]
    tabela["HC - gap"][situation] = ((tabela["HC - Melhor Solução"][situation] - otimos[situation]) / otimos[situation]) * 100

    tabela["(HC + VND) - Média Tempo"][situation] = statistics.mean(solucoes_time_HC_VND)
    tabela["(HC + VND) - Média solução"][situation] = statistics.mean(solucoes_HC_VND)
    tabela["(HC + VND) - Melhor Solução"][situation] = sorted(solucoes_HC_VND)[0]
    tabela["(HC + VND) - gap"][situation] = ((tabela["(HC + VND) - Melhor Solução"][situation] - otimos[situation]) / otimos[situation]) * 100

100%|████████████████████████████████████████████| 8/8 [00:19<00:00,  2.38s/it]


In [15]:
tabela

,otimo,HC - Média solução,HC - Melhor Solução,HC - Média Tempo,HC - gap,(HC + VND) - Média solução,(HC + VND) - Melhor Solução,(HC + VND) - Média Tempo,(HC + VND) - gap
P-n16-k8,450,538.6,468,0.0041029,4,528.7,468,0.0155113,4
P-n19-k2,212,393.5,317,0.0063043,49.5283,257.4,230,0.172113,8.49057
P-n20-k2,216,402.4,350,0.0049026,62.037,263.1,248,0.220149,14.8148
P-n23-k8,529,722.6,678,0.0083051,28.1664,695.2,649,0.0273162,22.6843
P-n55-k7,568,1093.1,1038,0.0464307,82.7465,868.5,774,0.521448,36.2676
P-n51-k10,741,1325,1231,0.0353239,66.1269,1154.6,1054,0.177518,42.2402
P-n50-k10,696,1231.3,1146,0.0487314,64.6552,1069.8,957,0.233353,37.5
P-n45-k5,510,1000.6,881,0.0316222,72.7451,760.1,674,0.417879,32.1569
